In [1]:
import sparknlp
spark = sparknlp.start()

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/tanvirbasi/.ivy2/cache
The jars for the packages stored in: /Users/tanvirbasi/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eaa2f7c5-1a6e-464e-81a4-7a982515f602;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.1.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.project

22/08/26 22:02:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/26 22:02:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.types import IntegerType
data = spark.read.csv("prelemmas.csv", header=True)

In [3]:
from sparknlp.pretrained import PretrainedPipeline
from sparknlp import Finisher
from pyspark.ml import Pipeline

In [4]:
finisher = Finisher().setInputCols(["lemmas"])
explain_pipeline_model = PretrainedPipeline("explain_document_ml").model

pipeline = Pipeline() \
    .setStages([
        explain_pipeline_model,
        finisher
        ])

model = pipeline.fit(data.select('id','label','text'))
annotations_finished_df = model.transform(data.select('id','label', 'text'))

explain_document_ml download started this may take some time.
Approx size to download 9.2 MB
[ / ]explain_document_ml download started this may take some time.
Approximate size to download 9.2 MB
Download done! Loading the resource.
[ — ]

[ \ ]

[ \ ]

[ | ]

[OK!]


In [5]:
from pyspark.sql.types import BooleanType
text_lemmas = annotations_finished_df.select('id', 'label', 'finished_lemmas')

In [7]:
from pyspark.sql import Row
'''
Make lemmas writable to CSV
'''
output = text_lemmas.rdd.map(lambda row: Row(id=row['id'], label=row['label'], text='|'.join(row['finished_lemmas'])))
output_df = output.toDF()
output_df.coalesce(1).write.csv('lemma.csv', header=True)

22/08/27 06:36:09 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3600320 ms exceeds timeout 120000 ms
22/08/27 06:36:09 WARN SparkContext: Killing executors is not supported by current scheduler.
